<a href="https://colab.research.google.com/github/AguaClara/SWOT/blob/master/Unit_Process_Designs/CDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chemical Dose Controller
The goal is to define the distinct design strategies required as a function of flow rate for the chemical dose controller. There are expected to be 3 distinct design regions.

Chemical resistance is critical for the design of the components exposed to chlorine. According to [Chemical Resistance of Thermoplastics Piping Materials](https://plasticpipe.org/pdf/tr-19_thermoplastic_pipe_for_transport_of_chemical.pdf), the plastics that are compatible with bleach at 12% active $Cl_2$ at temperatures above 40 $^\circ C$ are:
*   CPVC (chlorinated polyvinyl chloride)
*   PVC (polyvinyl chloride)
*   PP (polypropylene)
*   PB (polybutylene)


Arrays of multiple tubes each with their own valve are difficult to maintain because each tube needs to be removed in turn to be cleaned. To reduce the time required for cleaning it is preferred that a complete set of tubes be removed together. Thus this design will always have two flow resistors of equal flow that are installed in parallel with only one active at a time. The plant should have a third flow resistor on hand as a spare that can be clean and ready to replace a defective unit at any time.




1.   single dosing tube of 1/8" ID where the stock solution is varied to obtain a range of maximum chemical mass rates. A spare tube is plumbed and the active tube is selected by valves.
2.   set of multiple tubes with one valve at each end for the entire set.



In [ ]:
!pip install aguaclara

In [3]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

$$L_{Min} = \left( \frac{g h_L \pi D^4}{128 \nu Q_{Max}} - \frac{Q_{Max}}{16 \pi \nu} \sum{K} \right)$$

In [25]:
plantQ = 1 * u.L/u.s
pacDoseC = 40 * u.mg/u.L 
pacStockC = 1 * u.gram/u.L
pacQ = (plantQ * pacDoseC/pacStockC).to(u.mL/u.s)

temp = 15 * u.degC

K = 2
errorPI = 0.1
HL = 20 * u.cm
ID = 1/8 * u.inch

def tubeL(Q, HL,ID,errorPI,K,nu):
  return ((HL * u.gravity * np.pi * ID ** 4 / (128 * nu * Q)) - (8 * Q * K /(128 * np.pi * nu))).to(u.m)

def V_max(HL,errorPI,K):
  return (np.sqrt(2 * HL * u.gravity * errorPI/K)).to(u.mm/u.s)

def Q_max(HL,ID, errorPI,K):
  return (V_max(HL,errorPI,K) * ac.area_circle(ID)).to(u.mL/u.s)


def pacl_nu(pacStockC,temp):
  return (1 + ((2.383 * 10 ** -5) * (pacStockC.to(u.kg/u.m**3)).magnitude ** 1.893)) * ac.viscosity_kinematic_water(temp)

print("The target flow of PAC given the standard stock concentration is",pacQ)

print("The tube length to achieve the coagulant flow is", tubeL(pacQ, HL,ID,errorPI,K,pacl_nu(pacStockC,temp)))
print("The maximum flow for one tube is", Q_max(HL,ID, errorPI,K))
print("The tube length to achieve the max coagulant flow is", tubeL(Q_max(HL,ID, errorPI,K), HL,ID,errorPI,K,pacl_nu(pacStockC,temp)))

The target flow of PAC given the standard stock concentration is 40 milliliter / second
The tube length to achieve the coagulant flow is -1.292 meter
The maximum flow for one tube is 3.506 milliliter / second
The tube length to achieve the max coagulant flow is 1.104 meter


What would happen if we standardized on the length of the dosing tubes? The tube needs to be long enough to meet the error condition for water at the maximum temperature and the maximum head loss.

In [ ]:
tubeL = 1.2 * u.m
